<a href="https://colab.research.google.com/github/andcore20/Fifteen_Hundred_Curso_Dados/blob/main/pyspark_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **PYSPARK ON COLAB**



1. Instalar Dependências (3.1)

In [ ]:
!apt-get update -qq

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

!tar xf spark-3.1.2-bin-hadoop2.7.tgz

!pip install -q findspark

2. Declarando Variáveis de Ambiente

In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

3. Encontrar o Spark no Ambiente Colab

In [ ]:
import findspark

findspark.init()

4. Criando uma SparkSession

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
  .master('local[*]')\
  .appName("Iniciando com Spark")\
  .getOrCreate()

5. Leitura CSV em Spark

In [ ]:
#path = '/content/drive/MyDrive/data_sources/banco_central/credito_direcionado/indicador_custo.csv'

#df = spark.read.format("csv").option("header", "true").option("sep", ";").load(path)

## Começo

**1.** Trazer as bases de calendário, feriados nacionais e a de detran mensal

In [ ]:
from pyspark.sql.functions import *
from datetime import date

from google.colab import drive
drive.mount('/content/drive') #conectar drive com colab para importar .csv

path_staging = "/content/drive/MyDrive/17.UNI1500th/"

path_calendario = path_staging + "Calendario.csv"
path_feriados = path_staging + "Feriadosnacionais.csv"
path_detran_m = path_staging + "Detranmensal.csv"

df_calendario = spark.read.format("csv").option("header", "true").option("sep", ",").load(path_calendario) #format e option faz parte do metodo para usar pyspark no colab
df_feriados = spark.read.format("csv").option("header", "true").option("sep", ",").load(path_feriados)
df_detran_m = spark.read.format("csv").option("header", "true").option("sep", ",").load(path_detran_m)

#df_calendario = spark.read \
#    .format("csv") \
#    .option("header", "true") \
#    .load(path_calendario)
#df_feriados = spark.read \
#    .format("csv") \
#    .option("header", "true") \
#    .load(path_feriados)
#df_detran_m = spark.read \
#    .format("csv") \
#    .option("header", "true") \
#    .load(path_detran_m)


Mounted at /content/drive


In [ ]:
df_calendario.show()
df_feriados.show()
df_detran_m.show()

+--------------------+--------+
|             column1| column2|
+--------------------+--------+
|2016-06-30 00:00:...|00:00:00|
|2016-07-01 00:00:...|00:00:00|
|2016-07-02 00:00:...|00:00:00|
|2016-07-03 00:00:...|00:00:00|
|2016-07-04 00:00:...|00:00:00|
|2016-07-05 00:00:...|00:00:00|
|2016-07-06 00:00:...|00:00:00|
|2016-07-07 00:00:...|00:00:00|
|2016-07-08 00:00:...|00:00:00|
|2016-07-09 00:00:...|00:00:00|
|2016-07-10 00:00:...|00:00:00|
|2016-07-11 00:00:...|00:00:00|
|2016-07-12 00:00:...|00:00:00|
|2016-07-13 00:00:...|00:00:00|
|2016-07-14 00:00:...|00:00:00|
|2016-07-15 00:00:...|00:00:00|
|2016-07-16 00:00:...|00:00:00|
|2016-07-17 00:00:...|00:00:00|
|2016-07-18 00:00:...|00:00:00|
|2016-07-19 00:00:...|00:00:00|
+--------------------+--------+
only showing top 20 rows

+--------------------+-------------+--------------------+
|                Data|Dia_da_Semana|             Feriado|
+--------------------+-------------+--------------------+
|2001-01-01 00:00:...|segunda-fe

2. Tratar os dados das tabelas de calendário e feriados nacionais

In [ ]:
#padronizando formato das datas em calendario e feriado
df_calendario_v2 = df_calendario.select(to_date(df_calendario.column1).alias('Data_calendario')).collect() # transformando coluna "column1" em data e renomeando
df_feriados_v2 = df_feriados.withColumn("Data", regexp_replace(col("Data"),"00:00:00.0000000","")) # tirando parte da informação da coluna data
#df_feriados_v2 = df_feriados_v2.select(to_date(df_feriados_v2.Data).alias('Data')).collect()

#df_calendario_v2.show()
df_feriados_v2.show()

+-----------+-------------+--------------------+
|       Data|Dia_da_Semana|             Feriado|
+-----------+-------------+--------------------+
|2001-01-01 |segunda-feira|Confraternização ...|
|2001-02-26 |segunda-feira|            Carnaval|
|2001-02-27 |  terça-feira|            Carnaval|
|2001-04-13 |  sexta-feira|    Paixão de Cristo|
|2001-04-21 |       sábado|          Tiradentes|
|2001-05-01 |  terça-feira|     Dia do Trabalho|
|2001-06-14 | quinta-feira|      Corpus Christi|
|2001-09-07 |  sexta-feira|Independência do ...|
|2001-10-12 |  sexta-feira|Nossa Sr.a Aparec...|
|2001-11-02 |  sexta-feira|             Finados|
|2001-11-15 | quinta-feira|Proclamação da Re...|
|2001-12-25 |  terça-feira|               Natal|
|2002-01-01 |  terça-feira|Confraternização ...|
|2002-02-11 |segunda-feira|            Carnaval|
|2002-02-12 |  terça-feira|            Carnaval|
|2002-03-29 |  sexta-feira|    Paixão de Cristo|
|2002-04-21 |      domingo|          Tiradentes|
|2002-05-01 | quarta

# **Parte 2**

1. Precisamos criar uma coluna na tabela DETRANMENSAL que contenha a data completa

In [ ]:
# juntando colunas dia, mes, ano. E criando uma nova coluna para data
df_detran_m_v2 = df_detran_m.withColumn("Data", concat(col("ano"),lit("-"),col("mes"),lit("-"), col("dia") )).drop("dia").drop("ano").drop("mes") # da para fazer itens 1 e 2 juntos com select()

# transformando a coluna em tipo data pra ficar mais correta
df_detran_m_v2 = df_detran_m_v2.withColumn("Data", to_date(df_detran_m_v2.Data))

2. Devemos ter apenas as seguintes colunas na DETRANMENSAL: MARCA, MODELO, DATA e VIN

In [ ]:
# tirando coluna versao
df_detran_m_v2 = df_detran_m_v2.drop("versao")

df_detran_m_v2.show()

+----------+----------+-----------------+----------+
|     marca|    modelo|              vin|      Data|
+----------+----------+-----------------+----------+
|   PEUGEOT|      2008|VR3UDYHZSNJ526248|2022-01-06|
| CHEVROLET|      SAIL|LSGHD52H3ND024627|2022-01-06|
|   PEUGEOT|   PARTNER|VR3EF9HPANJ531887|2022-01-04|
| CHEVROLET|   CAPTIVA|LZWADAGA7NB024303|2022-01-24|
| CHEVROLET|  N400 MAX|LZWCDAGA3NC807556|2022-01-03|
|      FORD|NEW RANGER|MPBUR2366NX359311|2022-01-06|
| CHEVROLET|    GROOVE|LZWADAGA8NG006685|2022-01-03|
| CHEVROLET|    GROOVE|LZWADAGA2NG005841|2022-01-03|
| CHEVROLET|    GROOVE|LZWADAGA7NG006144|2022-01-03|
| CHEVROLET|    GROOVE|LZWADAGA1NG007063|2022-01-03|
|   PEUGEOT|      2008|VR3USHNSSNJ525794|2022-01-03|
| CHEVROLET|   CAPTIVA|LZWADAGA6NB027919|2022-01-24|
| CHEVROLET|   CAPTIVA|LZWADAGAXNB027826|2022-01-13|
|   PEUGEOT|       208|VR3UPHMGBNT002023|2022-01-04|
|   PEUGEOT|       208|VR3UPHMGBNT001380|2022-01-06|
|   HYUNDAI|  ATOS AH2|MALAF51CANM163805|2022-

3. Queremos saber como foi a venda dos carros da marca PEUGEOUT durante os meses

In [ ]:
df_detran_m_v2.count()

44653

In [ ]:
df_detran_m_v2.dropDuplicates()
df_detran_m_v2.distinct()
df_detran_m_v2.count() # contagem antes e depois para verificar mudança

44653

In [ ]:
# venda dos carros da marca PEUGEOUT durante os meses
vendas_peugeout = df_detran_m_v2.where(col("marca") == "PEUGEOT") # pegando apenas marcas da PEUGEOUT
vendas_peugeout_mes = vendas_peugeout.select( "marca", month('Data').alias('mes')) # pegando o mes da coluna data
vendas_peugeout_mes_count = vendas_peugeout_mes.groupBy(vendas_peugeout_mes.mes).count()
vendas_peugeout_mes_count.sort(asc("mes")).show()


+---+-----+
|mes|count|
+---+-----+
|  1| 1499|
|  2| 1195|
|  3|  664|
|  4|  870|
|  5|  573|
|  6|  723|
|  7| 1008|
|  8|  909|
+---+-----+



4. Os modelos de veículo: GOL e HILUX estão incorretos. Na verdade, deveriam ser respectivamente UP e SW4;

In [ ]:
df_detran_m_v3 = df_detran_m_v2
GOL = df_detran_m_v3.where(col("modelo")=="GOL").count()
HILUX = df_detran_m_v3.where(col("modelo")=="HILUX").count()
UP = df_detran_m_v3.where(col("modelo")=="UP").count()
SW4 = df_detran_m_v3.where(col("modelo")=="SW4").count()

modelo_count = [{'GOL': GOL, 'HILUX': HILUX,'UP':UP,'SW4':SW4}]
print(modelo_count)

[{'GOL': 947, 'HILUX': 2265, 'UP': 0, 'SW4': 0}]


In [ ]:
df_detran_m_v3 = df_detran_m_v3.withColumn("modelo",regexp_replace(col("modelo"),"GOL","UP"))
df_detran_m_v3 = df_detran_m_v3.withColumn("modelo",regexp_replace(col("modelo"),"HILUX","SW4"))


In [ ]:
print(modelo_count)

[{'GOL': 947, 'HILUX': 2265, 'UP': 0, 'SW4': 0}]


5. Por se tratar de uma empresa de outro país, não consideram os feriados de CARNAVAL, INDEPÊNDENCIA DO BRASIL e NOSSA SENHORA APARECIDA.

In [ ]:
valores_feriado = df_feriados_v2.select(col("Feriado")).distinct().collect()
for valor in valores_feriado:
    print(valor[0])

Tiradentes
Proclamação da República
Nossa Sr.a Aparecida - Padroeira do Brasil
Dia do Trabalho
Corpus Christi
Natal
Confraternização Universal
Carnaval
Independência do Brasil
Paixão de Cristo
Finados


In [ ]:
df_feriados_v2.count()

936

In [ ]:
df_feriados_v3 = df_feriados_v2.where((col("Feriado") != "Carnaval") &
                                     (col("Feriado") != "Independência do Brasil") &
                                     (col("Feriado") != "Nossa Sr.a Aparecida - Padroeira do Brasil"))
df_feriados_v3.count()

624

In [ ]:
# Fazer um join da base de detran com a base de calendário (aqui vamos querer que o dataframe final contenha todos os dias que estão cadastrados na base calendário);

teste = df_detran_m_v3.join(df_feriados_v3, df_detran_m_v3.Data == df_feriados_v3.Data, 'outer').concat()

In [ ]:
teste2 = df_detran_m_v3.join(df_feriados_v3, df_detran_m_v3.Data == df_feriados_v3.Data, 'outer').select(
    df_detran_m_v3.Data, df_detran_m_v3.marca, df_feriados_v3.Feriado)

In [ ]:
teste2.show()

+----------+---------+--------------------+
|      Data|    marca|             Feriado|
+----------+---------+--------------------+
|      null|     null|               Natal|
|      null|     null|Proclamação da Re...|
|2022-03-28|CHEVROLET|                null|
|2022-03-28|CHEVROLET|                null|
|2022-03-28|CHEVROLET|                null|
|2022-03-28|CHEVROLET|                null|
|2022-03-28|CHEVROLET|                null|
|2022-03-28|CHEVROLET|                null|
|2022-03-28|CHEVROLET|                null|
|2022-03-28|CHEVROLET|                null|
|2022-03-28|CHEVROLET|                null|
|2022-03-28|CHEVROLET|                null|
|2022-03-28|   TOYOTA|                null|
|2022-03-28|  HYUNDAI|                null|
|2022-03-28|  HYUNDAI|                null|
|2022-03-28|  HYUNDAI|                null|
|2022-03-28|  PEUGEOT|                null|
|2022-03-28|CHEVROLET|                null|
|2022-03-28|CHEVROLET|                null|
|2022-03-28|  PEUGEOT|          

In [ ]:
teste.show()

+---------+-----------+-----------------+----------+-----------+-------------+--------------------+
|    marca|     modelo|              vin|      Data|       Data|Dia_da_Semana|             Feriado|
+---------+-----------+-----------------+----------+-----------+-------------+--------------------+
|     null|       null|             null|      null|2002-12-25 | quarta-feira|               Natal|
|     null|       null|             null|      null|2007-11-15 | quinta-feira|Proclamação da Re...|
|CHEVROLET|   N400 MAX|LZWCDAGA2NC811162|2022-03-28|       null|         null|                null|
|CHEVROLET|   N400 MAX|LZWCDAGA8NC812123|2022-03-28|       null|         null|                null|
|CHEVROLET|   N400 MAX|LZWCDAGA7NC812016|2022-03-28|       null|         null|                null|
|CHEVROLET|   N400 MAX|LZWCDAGA4NC811065|2022-03-28|       null|         null|                null|
|CHEVROLET|   N400 MAX|LZWCDAGA0NC812116|2022-03-28|       null|         null|                null|
